# Notebook to edit a device config
___To use this notebook...___
1. configure your username and password
2. configure IP or hostname of the device you want to configure
3. If you want to see some more DEBUG messages set loglevel to DEBUG (optional)

### define your login and set backup Directory

In [ ]:
username = "username"
password = "password"

### on which device do you want to change the configuration?

In [ ]:
device = "ip_address"

### set loglevel

In [ ]:
loglevel = "INFO"

### The directory where we store the configurations

In [ ]:
config_directory = "./device_configs"

# Basic setup .. import, logging and so on

In [ ]:
import sys
import difflib
import os
from loguru import logger
from veritas.devicemanagement import scrapli as dm
from IPython.display import display, HTML

In [ ]:
logger.configure(extra={"extra": "unset"})
logger.remove()
logger.add(sys.stderr, level=loglevel)

# Connect to device
Change platform, manufacturer and ssh port if neede

In [ ]:
conn = dm.Devicemanagement(ip=device,
                           platform="ios",
                           manufacturer="cisco",
                           username=username,
                           password=password,
                           port=22)
conn.open()

# get running-config

In [ ]:
running_config = conn.get_config('running')

## If you want to see the config run the next cell

In [ ]:
print(running_config)

## Write config to disk

In [ ]:
config_file = f'./{config_directory}/{device}.config'
directory = os.path.dirname(config_file)
if not os.path.exists(directory):
    logger.debug(f'creating {directory}')
    os.makedirs(directory)
with open(config_file, 'w') as f:
    f.write(running_config)

# Now open config in new tab and edit it. Don't forget to save it after you did your modifications.

# Update config

## load config before we replace it

In [ ]:
with open(config_file, 'r') as f:
   new_config = f.read()

### check if config is okay (optional)

In [ ]:
print(new_config)

## prepare session and upload config
replace=True means that the complete config will be replaced. ___If you only have a part of the configuration, you have to set the parameter to False.___

In [ ]:
conn.prepare()
conn.load_config(config=new_config, replace=True)

### show diff between running and your new config (optional)

In [ ]:
diff_result = conn.diff_config()
print(diff_result.device_diff)

## Everything correct? Than commit the config change

In [ ]:
conn.commit_config()

## otherwise abort the config change

In [ ]:
conn.abort_config()

## clean up

In [ ]:
conn.cleanup()